In [1]:
import pandas as pd
import requests
from datetime import date, timedelta
from ml_functions import get_rf_ensemble_plot, get_random_forest_df, get_historical_data
from skgarden import RandomForestQuantileRegressor
from plotly.io import write_html
import plotly.express as px
import numpy as np





In [3]:
end_date = date.today().isoformat()

In [ ]:
end_date

In [4]:
delta = timedelta(730)

In [5]:
start_date = (date.today() - delta).isoformat()

In [ ]:
start_date


In [2]:
url = 'https://api.exchangeratesapi.io/history'


In [6]:
payload = {
    "start_at": start_date,
    "end_at": end_date,
    "symbols": "CNY,JPY,EUR",
    "base": "USD"
}

In [7]:
result = requests.get(url, params = payload).json()

In [8]:
rate_df = pd.DataFrame(result['rates']).transpose()

In [9]:
rate_df = rate_df.sort_index()


In [10]:
rate_df.index = pd.to_datetime(rate_df.index, format = '%Y-%m-%d')

In [ ]:
rate_df

In [11]:
btc_price = get_historical_data('BTC', 730)

In [ ]:
len(btc_price)

In [12]:
btc_price = btc_price.rename(columns = {'close': 'BTC'})
    
    

In [ ]:
btc_price.head()

In [13]:
eth_price = get_historical_data('ETH', 730)

In [14]:
eth_price = eth_price.rename(columns = {'close': 'ETH'})

In [15]:
ltc_price = get_historical_data('LTC', 730)

In [16]:
ltc_price = ltc_price.rename(columns = {'close': 'LTC'})

In [17]:
coin_df = btc_price

In [18]:
coin_df['LTC'] = ltc_price

In [19]:
coin_df['ETH'] = eth_price

In [20]:
combined_df = pd.concat([coin_df, rate_df], axis = 1 )

In [21]:
combined_df.head()

,BTC,LTC,ETH,EUR,CNY,JPY
2018-01-21,11549.93,190.73,1049.09,NaN,NaN,NaN
2018-01-22,10814.52,179.42,999.64,0.817060,6.403628,110.842389
2018-01-23,10858.23,177.92,984.47,0.816393,6.405911,110.498816
2018-01-24,11429.02,180.59,1061.78,0.809585,6.372976,109.399288
2018-01-25,11175.87,178.97,1046.37,0.805997,6.331023,108.906263


In [22]:
combined_df.tail()

,BTC,LTC,ETH,EUR,CNY,JPY
2020-01-17,8894.83,60.97,169.98,0.900252,6.858660,110.109831
2020-01-18,8909.28,59.22,173.91,NaN,NaN,NaN
2020-01-19,8702.59,57.66,166.92,NaN,NaN,NaN
2020-01-20,8634.03,57.09,166.64,0.902120,6.866216,110.184935
2020-01-21,8655.97,57.78,167.57,NaN,NaN,NaN


In [23]:
combined_df['EUR'] = combined_df['EUR'].ffill()


In [24]:
combined_df.head()

,BTC,LTC,ETH,EUR,CNY,JPY
2018-01-21,11549.93,190.73,1049.09,NaN,NaN,NaN
2018-01-22,10814.52,179.42,999.64,0.817060,6.403628,110.842389
2018-01-23,10858.23,177.92,984.47,0.816393,6.405911,110.498816
2018-01-24,11429.02,180.59,1061.78,0.809585,6.372976,109.399288
2018-01-25,11175.87,178.97,1046.37,0.805997,6.331023,108.906263


In [25]:
combined_df['CNY'] = combined_df['JPY'].ffill()


In [26]:
combined_df['JPY'] = combined_df['CNY'].ffill()

In [ ]:
combined_df.tail()

In [ ]:
combined_df.head()

In [27]:
combined_df = combined_df.reset_index()

In [28]:
combined_df.head()

,index,BTC,LTC,ETH,EUR,CNY,JPY
0,2018-01-21,11549.93,190.73,1049.09,NaN,NaN,NaN
1,2018-01-22,10814.52,179.42,999.64,0.817060,110.842389,110.842389
2,2018-01-23,10858.23,177.92,984.47,0.816393,110.498816,110.498816
3,2018-01-24,11429.02,180.59,1061.78,0.809585,109.399288,109.399288
4,2018-01-25,11175.87,178.97,1046.37,0.805997,108.906263,108.906263


In [35]:
last_day = combined_df['index'].iloc[-1]

In [36]:
last_day

Timestamp('2020-01-21 00:00:00')

In [37]:
delta = timedelta(1)
add_row_date = (last_day + delta)

In [38]:
add_row_date

Timestamp('2020-01-22 00:00:00')

In [39]:
df2 = pd.DataFrame([[add_row_date,0,0,0,0,0,0]], columns= ['index','BTC','LTC','ETH','EUR','CNY','JPY'])

In [ ]:
combined_df = combined_df.append(df2, ignore_index = True)

In [ ]:
combined_df = combined_df.set_index('index')

In [ ]:
combined_df.tail()

In [ ]:
combined_df.tail()

In [ ]:
combined_df.tail()

In [ ]:
combined_df['BTC_Previous_Day'] = combined_df['BTC'].shift(1)

In [ ]:
combined_df.tail()

In [ ]:
combined_df['LTC_Previous_Day'] = combined_df['LTC'].shift(1)

In [ ]:
combined_df['ETH_Previous_Day'] = combined_df['ETH'].shift(1)

In [ ]:
combined_df['EUR_Previous_Day'] = combined_df['EUR'].shift(1)

In [ ]:
combined_df['CNY_Previous_Day'] = combined_df['CNY'].shift(1)

In [ ]:
combined_df['JPY_Previous_Day'] = combined_df['JPY'].shift(1)

In [ ]:
combined_df.tail()

In [ ]:
combined_df = combined_df.drop(columns = ['EUR', 'CNY', 'JPY'])

In [ ]:
combined_df = combined_df.dropna()

In [ ]:
combined_df.tail()

In [ ]:
new_df = combined_df.copy()

In [ ]:
new_df = get_random_forest_df()

In [ ]:
new_df

In [ ]:
len(new_df)


In [ ]:
X = new_df.drop(columns = ['BTC', 'LTC', 'ETH'])


In [ ]:
X.tail()

In [ ]:
y = new_df['BTC']

In [ ]:
X_train = X[:725]


In [ ]:
y_train = y[:725]

In [ ]:
X_test = X[725:]

In [ ]:
y_test = y[725:]

In [ ]:
regressor = RandomForestQuantileRegressor(random_state = 0, n_estimators = 500)

In [ ]:
model = regressor.fit(X_train, y_train)

In [ ]:
model.score(X_train, y_train)

In [ ]:
model.score(X_test,y_test)

In [ ]:
print(model.feature_importances_)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
predictions

In [ ]:
actuals = pd.DataFrame(y_test)

In [ ]:
actuals_vs_predictions = actuals.copy()

In [ ]:
actuals_vs_predictions['Predictions'] = predictions

In [ ]:
actuals_vs_predictions = actuals_vs_predictions.reset_index()

In [ ]:
actuals_vs_predictions

In [ ]:
fig = px.line(actuals_vs_predictions, x = 'index', y = 'Predictions')
fig.update_layout(autosize = True, height = 800, width = 950, title_text = 'BTC Random Forest Ensemble', template = 'seaborn')
fig.update_xaxes(nticks = 5, title = 'Date')
fig.update_yaxes(automargin=True)

In [ ]:
get_rf_ensemble_plot()